# Tp Ciencia de datos 2025

## Grupo 2
**Integrantes:**
- Francisco Lucich
- Esteban Luna
- Francisco Moretti
- Tomás Zubik


# Carga del dataset

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# Seleccionamos 'all' para obtener tanto el set de entrenamiento como el de test
# Removemos headers, footers y quotes para simplificar el texto
newsgroups_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [ ]:
# Una lista donde cada elemento es el texto de un documento
newsgroups_data.data

In [ ]:
# Un array de NumPy con números enteros que representan la categoría de cada documento.
newsgroups_data.target

In [ ]:
# Una lista con los nombres correspondientes a cada número en .target.
newsgroups_data.target_names

# Fase 1: Carga, Exploración Inicial y Limpieza Básica

**1: carga del dataset**

In [ ]:
import pandas as pd
# Crear el DataFrame
df = pd.DataFrame({'texto': newsgroups_data.data, 'target': newsgroups_data.target})
# Mapear los números de target a los nombres de las categorías
df['categoria'] = df['target'].apply(lambda i: newsgroups_data.target_names[i])
# Ahora puedes descartar la columna 'target' si lo deseas
# df = df.drop('target', axis=1)
print(df.head())
print(f"Nombres de las categorías: {newsgroups_data.target_names}")

**2: Exploracion inicial**

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# df.inspect() no existe result: AttributeError: 'DataFrame' object has no attribute 'inspect''ArithmeticError

# describe() muestra estadísticas descriptivas de las columnas numéricas del DataFrame,
# como media, desviación estándar, mínimos, máximos y percentiles.
df.describe()

**2.1 analisis de categorias**

In [ ]:
# La función value_counts() cuenta la cantidad de ocurrencias de cada valor en una columna.
#sort_index() ordena los resultados por el índice (en este caso, las categorías).
df.value_counts('categoria').sort_index()

In [ ]:
# Visualización: Gráfico de barras de la distribución de documentos por categoría
import matplotlib.pyplot as plt
import seaborn as sns

# Contar documentos por categoría
conteo_categorias = df['categoria'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
sns.barplot(x=conteo_categorias.index, y=conteo_categorias.values, palette="viridis")
plt.xticks(rotation=90, ha='center')
plt.yticks(range(0, conteo_categorias.max() + 100, 100))
plt.xlabel('Categoría')
plt.ylabel('Cantidad de documentos')
plt.title('Distribución de documentos por categoría')
plt.tight_layout()
plt.show()

**3. limpieza basica**

In [ ]:
import string
df['texto_limpio'] = df['texto'].str.lower()  # Convertir a minúsculas

df['texto_limpio'] = df['texto_limpio'].str.replace(r'\d+', '', regex=True)  # Eliminar números

translator = str.maketrans('', '', string.punctuation)  # Crear un traductor para eliminar puntuación
df['texto_limpio'] = df['texto_limpio'].apply(lambda x: x.translate(translator))  # Eliminar puntuación

df['texto_limpio'] = df['texto_limpio'].str.replace(r'\s+', ' ', regex=True)  # Eliminar espacios en blanco


In [ ]:
df['texto_limpio'].head()

# Fase 2: Preprocesamiento con NLTK

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

**1. Tokenizacion**

In [ ]:
df['tokens']= df['texto_limpio'].apply(nltk.word_tokenize)  # Tokenización

df['tokens']

**2. Eliminacion de Stop words**

In [ ]:
nltk.corpus.stopwords.words('english')

In [ ]:
df['tokens_sin_stopwords'] = df['tokens'].apply(lambda x: [word for word in x if word not in nltk.corpus.stopwords.words('english')])
df['tokens_sin_stopwords'].head()

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords_personalizadas = {'would', 'could', 'also', 'one', 'get', 'like', 'use', 'subject', 'writes', 'article'}

# Unir ambas listas
stopwords_actualizadas = stopwords.union(stopwords_personalizadas)

# Aplicar la eliminación de stopwords (incluyendo las personalizadas)
df['tokens_sin_stopwords'] = df['tokens'].apply(lambda x: [word for word in x if word not in stopwords_actualizadas])

**2. Lematización**

In [ ]:
from nltk.stem import WordNetLemmatizer

# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Aplicar lematización
df['tokens_lematizados'] = df['tokens_sin_stopwords'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

df[['tokens_sin_stopwords', 'tokens_lematizados']].head()


*Comparación Lematización y Stemming*

La lematización reduce las palabras a su forma base (lema) utilizando un enfoque basado en el significado y la gramática. Es más preciso pero más lento. Mientras que el stemming recorta las palabras a su raíz básica sin considerar el contexto gramatical. Es más rápido pero menos preciso.

Usaríamos lemmatización cuando se necesita precisión y el significado de las palabras es importante (por ejemplo, análisis semántico), y stemming cuando se valora más la  velocidad y no importa perder algo de precisión (por ejemplo, motores de búsqueda).

**4. Análisis de Tokens**

In [ ]:
from nltk.probability import FreqDist

# Combinar todos los tokens lematizados en una sola lista

todos_los_tokens = [token for tokens in df['tokens_lematizados'] for token in tokens]

# Calcular la frecuencia de cada token
frecuencia_tokens = FreqDist(todos_los_tokens)

tokens_mas_frecuentes = frecuencia_tokens.most_common(30)
print("Tokens más frecuentes:", tokens_mas_frecuentes)

In [ ]:

# Preparar datos para el gráfico
tokens, frecuencias = zip(*tokens_mas_frecuentes)

plt.figure(figsize=(12, 6))
sns.barplot(x=list(tokens), y=list(frecuencias), palette="viridis")
plt.xticks(rotation=45, ha='right')
plt.xlabel('Tokens')
plt.ylabel('Frecuencia')
plt.title('Tokens más frecuentes después del preprocesamiento')
plt.tight_layout()
plt.show()


In [ ]:
# Crear una función para calcular frecuencias por categoría
def frecuencia_por_categoria(categoria):
    tokens_categoria = [token for tokens in df[df['categoria'] == categoria]['tokens_lematizados'] for token in tokens]
    return FreqDist(tokens_categoria)

# Seleccionar algunas categorías para análisis
categorias_seleccionadas = df['categoria'].unique()[:5] 

for categoria in categorias_seleccionadas:
    frecuencia_categoria = frecuencia_por_categoria(categoria)
    tokens_categoria, frecuencias_categoria = zip(*frecuencia_categoria.most_common(10))
    
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(tokens_categoria), y=list(frecuencias_categoria), palette="viridis")
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Tokens')
    plt.ylabel('Frecuencia')
    plt.title(f'Tokens más frecuentes en la categoría: {categoria}')
    plt.tight_layout()
    plt.show()